## Importación de librerías

In [19]:
import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.proportion import proportions_ztest
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

## Carga de datos `df_final_eda`

In [2]:
df = pd.read_parquet("../data/2.processed/df_final_eda.parquet")

## Intervalos de confianza

In [3]:
orden = ["early morning", "morning", "afternoon", "evening", "night", "late night"]
df["moment_of_day"] = pd.Categorical(df["moment_of_day"], categories=orden, ordered=True)

group = df.groupby("moment_of_day", observed = True)["is_fraud"].agg(["count", "sum"]).reset_index()
group.columns = ["moment_of_day", "nº_transactions", "nº_frauds"]

group["rate"] = (group["nº_frauds"] / group["nº_transactions"]) * 100

low, high = proportion_confint(count=group["nº_frauds"], nobs=group["nº_transactions"], alpha=0.05, method="wilson")

group["ci_low"] = low * 100
group["ci_high"] = high * 100

group

,moment_of_day,nº_transactions,nº_frauds,rate,ci_low,ci_high
0,early morning,31604,44,0.139223,0.103733,0.186833
1,morning,42101,44,0.104511,0.077866,0.140260
2,afternoon,82318,105,0.127554,0.105389,0.154374
3,evening,65886,70,0.106244,0.084109,0.134197
4,night,52790,671,1.271074,1.178997,1.370243
5,late night,50391,967,1.918993,1.802823,2.042495


En este apartado se evalúa si las diferencias observadas en el análisis descriptivo responden a patrones consistentes o simplemente se deben a la variabilidad de la muestra, para ello se calculan los intervalos de confianza al **95%** para las tasas de fraude por franja horaria `moment_of_day`, utilizando el método de **Wilson**.

Los resultados muestran que la franja nocturna presenta una tasa de fraude superior a la media global con **1,27** en **night** y un **1,91** en **late night**, sus intervalos de confianza no se solapan con el de las franjas de menor riesgo diurnas, lo que indica que la mayor incidencia producida en la franja nocturna no puede atribuirse al azar, sino que refleja un patrón consistente asociado al momento del día.

Asimismo, se observa que los intervalos son más amplios en segmentos con menor volumen de transacciones, lo que confirma la importancia de considerar el tamaño muestral al interpretar tasas elevadas.

## Tests de diferencia de proporciones

### **Noche vs Día**

In [4]:
night = df[(df["hour"] >= 22) | (df["hour"] <= 3)]
day = df[(df["hour"] >= 9) & (df["hour"] <= 18)]

count = np.array([night["is_fraud"].sum(), day["is_fraud"].sum()])
nobs = np.array([len(night), len(day)])

z, p = proportions_ztest(count, nobs)
p_night = count[0] / nobs[0]
p_day = count[1] / nobs[1]
diff = p_night - p_day

print(f"Noche: {p_night*100:.3f}% ({count[0]}/{nobs[0]})", "\n")
print(f"Día:   {p_day*100:.3f}% ({count[1]}/{nobs[1]})", "\n")
print(f"Diferencia: {diff*100:.3f} pp", "\n")
print(f"z = {z:.3f}, p = {p:.3e}")

Noche: 2.122% (1612/75961) 

Día:   0.122% (180/146981) 

Diferencia: 2.000 pp 

z = 50.115, p = 0.000e+00


Para validar la diferencia entre las franjas horarias, se realiza un **test z** de diferencia de proporciones comparando, un grupo nocturno que comprende el horario de **22:00** a **03:00** y un grupo diurno que comprende de **09:00** a **18:00**.

La tasa de fraude en el horario nocturno presenta un valor de **2,122%** aproximadamente **17** veces superior a la observada durante el día con un valor de **0,122%**.

El contraste mostro un **z = 50,115** y un **p-valor < 0,001**, dado que el **p-valor** es inferior al nivel de significación del **5%**, se rechaza la hipótesis nula de igualdad de proporciones. Por tanto, existe evidencia estadística de que la probabilidad de fraude es significativamente mayor en el horario nocturno.

Además, ambos grupos cuentan con tamaños muestrales elevados, más de **75.000** observacaiones en noche y más de **146.000** observaciones en día, lo que refuerza el resultado y reduce la probabilidad de que la diferencia observada se deba a variabilidad muestral.

La franja nocturna constituye un segmento de riesgo claramente diferenciado, lo que justifica su priorización en el sistema de monitorización y la implementación de reglas operativas específicas para transacciones realizadas entre las **22:00** y las **03:00**.

### **Top categorías vs resto**

In [17]:
cat = df.groupby("category", observed= True)["is_fraud"].agg(n="count", frauds="sum")
cat["rate"] = cat["frauds"] / cat["n"]

min_n = 500
top_c = 5

top_cats = ( cat[cat["n"] >= min_n].sort_values("rate", ascending=False).head(top_c).index.tolist())

top_cats = list(top_cats)
top_cats

['Shopping_net', 'Misc_net', 'Grocery_pos', 'Shopping_pos', 'Gas_transport']

Con el objetivo de identificar las categorías con mayor riesgo de fraude, se realiza una agregación por variable category, calculando para cada grupo el número total de transacciones **n**, el número de fraudes **frauds** y la tasa de fraude correspondiente **rate = frauds / n**.

Para evitar estimaciones inestables con bajo soporte muestral, se establece un umbral mínimo de **500** transacciones por categoría. Posteriormente, se ordenan las categorías por tasa de fraude y se seleccionaron las **5** con mayor incidencia.

El resultado es un conjunto de **Top categorías** definido de manera objetiva basado en dos criterios, mayor tasa observada y volumen mínimo suficiente para interpretaciones más robustas. Estas categorías serán comparadas con el conjunto restante mediante un test de diferencia de proporciones.

In [9]:
top_df = df[df["category"].isin(top_cats)]
rest_df = df[~df["category"].isin(top_cats)]

A partir del conjunto **top_cats**, categorías seleccionadas como las de mayor tasa de fraude, se divide el dataset en dos grupos para poder compararlos estadísticamente. 

El primer grupo **top_df** contiene únicamente las transacciones cuya categoría pertenece a **top_cats**, es decir, las operaciones asociadas a las categorías de mayor riesgo. El segundo grupo **rest_df** incluye el resto de transacciones, correspondientes a todas las categorías no incluidas en ese top. Esta partición permite realizar un contraste formal **Top categorías vs resto** y evaluar si la diferencia de tasas observada es robusta.

In [18]:
count = np.array([top_df["is_fraud"].sum(), rest_df["is_fraud"].sum()])
nobs = np.array([len(top_df), len(rest_df)])

z, p = proportions_ztest(count, nobs)

p_top = count[0] / nobs[0]
p_rest = count[1] / nobs[1]
diff = p_top - p_rest

print(f"Top categorías (K={top_c}, min_n={min_n}): {p_top*100:.3f}% ({count[0]}/{nobs[0]})", "\n")
print(f"Resto categorías:                 {p_rest*100:.3f}% ({count[1]}/{nobs[1]})", "\n")
print(f"Diferencia: {diff*100:.3f} pp", "\n")
print(f"z = {z:.3f}, p = {p:.3e}", "\n")
print("Top categorías usadas:", top_cats)

Top categorías (K=5, min_n=500): 1.091% (1446/132519) 

Resto categorías:                 0.236% (455/192571) 

Diferencia: 0.855 pp 

z = 31.414, p = 1.295e-216 

Top categorías usadas: ['Shopping_net', 'Misc_net', 'Grocery_pos', 'Shopping_pos', 'Gas_transport']


Con el fin de evaluar si las categorías identificadas como de mayor riesgo presentan una incidencia de fraude superior al resto, se realiza un test de diferencia de proporciones entre ambos grupos.

En primer lugar, se contabiliza el número total de fraudes y el número total de transacciones en el grupo de **Top categorías** y en el grupo **restante**, a partir de estos valores se calculan las tasas de fraude correspondientes a cada conjunto. La tasa de fraude en las categorías de mayor riesgo es **1,091%** aproximadamente **4,6** veces superior a la observada en el resto de **0,236%**.

Posteriormente, se aplica un contraste **z** para dos proporciones independientes, tomando como hipótesis nula la igualdad de tasas entre ambos grupos, con un estadístico **z** elevado de **31.414** y un **p-valor** bastante reducido **< 0,001** nos permite rechazar la hipótesis nula de igualdad de proporciones. Por tanto, existe evidencia estadística de que las categorías seleccionadas presentan una incidencia de fraude significativamente superior al resto.

Además de la significación del estadístico **z** y del **p-valor**, se calcula la diferencia absoluta en puntos porcentuales entre ambas tasas, con el objetivo de cuantificar el tamaño del efecto observado.

Este procedimiento permite determinar no solo si las categorías seleccionadas presentan mayor riesgo, sino también si dicha diferencia es estadísticamente robusta y relevante en términos prácticos. Además, ambos grupos cuentan con tamaños muestrales elevados más de **130.000** observaciones para el grupo de **Top categorías** y más de **190.000** observaciones para el **resto** de categorías, lo que refuerza la estabilidad de la estimación y minimiza la probabilidad de que el resultado sea atribuible a variabilidad muestral.

## Baseline: Regresión Logística simple

### **ROC-AUC**

In [7]:
X = df[["amt_vs_avg_agi","hour"]]
y = df["is_fraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

model = LogisticRegression(class_weight="balanced", max_iter=2000)
model.fit(X_train, y_train)

proba = model.predict_proba(X_test)[:,1]

roc_auc_score(y_test, proba)

0.8461713087136293

Para observar la capacidad explicativa conjunta de las variables identificadas en el **EDA**, se realiza un modelo de regresión logística binaria utilizando como predictores, **amt_vs_avg_agi** y **hour**. 

Se seleccionan estas dos variables por su coherencia con los hallazgos obtenidos en el análisis previo. El importe absoluto de la transacción no resulta tan informativo como su valor relativo en función del contexto socioeconómico, además el análisis estadístico demuestra que la tasa de fraude en el horario nocturno es superior a la diurna, con diferencias estadísticamente significativas, por estos motivos se decide utilizar estas dos variables en la estimación del modelo.

Se utiliza una partición de entrenamiento y test de **70%** y **30%** respectivamente con el fin de mantener un conjunto de evaluación suficientemente amplio y estable. Dado el tamaño total del dataset, el **70%** proporciona volumen suficiente para el aprendizaje del modelo, mientras que el **30%** permite una estimación robusta del rendimiento fuera de muestra. Además, se emplea el parámetro **class_weight="balanced"** para evitar que el modelo favorezca sistemáticamente la clase mayoritaria.

El rendimiento del modelo se evaluó mediante el área bajo la curva ROC, obteniendo un **ROC_AUC** de **0.846** lo que indica una capacidad discriminativa elevada para un modelo simple con únicamente dos variables. Este resultado sugiere que el importe relativo **amt_vs_avg_agi** y **hour**, la hora de la transacción concentran una parte relevante de la capacidad predictiva asociada al fraude.

### **PR-AUC**

In [21]:
pr_auc = average_precision_score(y_test, proba)

print(f"PR-AUC (Average Precision): {pr_auc:.3f}")

PR-AUC (Average Precision): 0.168


La **PR-AUC** es una métrica más exigente utilizada en escenarios desbalanceados, en este caso muestra un valor de **0,168**, aunque el fraude es un evento poco frecuente, el modelo consigue concentrar una fracción significativa de los casos positivos en los rangos de mayor probabilidad. Esto sugiere que el baseline puede utilizarse para priorizar revisión manual, mejorando la eficiencia respecto a una selección aleatoria.

En conjunto, los resultados confirman que las variables seleccionadas aportan capacidad predictiva real, manteniendo coherencia con el objetivo de validación estadística del proyecto.